# Analise de banlist

- Objetivo: criar uma funcao correspondente ao cod da carta

In [210]:
import re
import numpy as np
import pandas as pd

In [211]:
# csv da banlist baixado
banlist = '/home/erick/Documents/DeckAPI/src/var/Home.csv'
complet = '/home/erick/Documents/DeckAPI/src/cache/complet.csv'

df_ban = pd.read_csv(banlist, sep='|', skipinitialspace=True)
df_com = pd.read_csv(complet, sep=';', skipinitialspace=True)

In [212]:
df_ban = df_ban.loc[~df_ban['card_name'].isin(['Card Name'])].reset_index(drop=True)
df_ban

,card_type,card_name,condition,remarks
0,Monster/Effect,AGIDO THE ANCIENT SENTINEL,Forbidden,NaN
1,Monster/Effect,"ALPHA, THE MASTER OF BEASTS",Forbidden,NaN
2,Monster/Effect,AMANO-IWATO,Forbidden,NaN
3,Monster/Effect,APOQLIPHORT TOWERS,Forbidden,NaN
4,Monster/Effect,ARCHLORD KRISTYA,Forbidden,NaN
...,...,...,...,...
547,Trap,CYBERNETIC OVERFLOW,Unlimited,NaN
548,Trap,MAGICAL EXPLOSION,Unlimited,NaN
549,Trap,ULTIMATE CRYSTAL MAGIC,Unlimited,NaN
550,Trap,VIRTUAL WORLD GATE - CHUCHE,Unlimited,NaN


In [213]:
df_com

,cod,card_name,tipo,effect,arquetype
0,2511,Labrynth Cooclock,[Effect / Monster],(Quick Effect): You can discard this card; thi...,Labrynth
1,10000,Ten Thousand Dragon,[Effect / Monster],Cannot be Normal Summoned/Set. Must be Special...,generic
2,27551,Limit Reverse,Trap,Target 1 monster with 1000 or less ATK in your...,generic
3,32864,The 13th Grave,[Normal / Monster],A zombie that suddenly appeared from plot #13 ...,generic
4,35699,SPYRAL Sleeper,[Effect / Monster],Cannot be Normal Summoned/Set. Must be Special...,SPYRAL
...,...,...,...,...,...
13275,501000002,King of Destruction - Xexex,[Effect / Monster],This card cannot be Special Summoned. This car...,generic
13276,501000003,Queen of Fate - Eternia,[Effect / Monster],This card cannot be Special Summoned. This car...,generic
13277,501000004,Testament of the Arcane Lords,[Effect / Monster],This card cannot be Special Summoned. This car...,generic
13278,501000006,"Chimaera, the Master of Beasts",[Effect / Monster],This card cannot be Special Summoned. To Tribu...,generic


In [214]:
df_ban['card_name'] = df_ban['card_name'].str.strip().str.upper()
df_ban['card_name'] = df_ban['card_name'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

In [215]:
cod_name = df_com[['cod', 'card_name']].copy()
cod_name['card_name'] = cod_name['card_name'].str.upper().str.strip()
cod_name

,cod,card_name
0,2511,LABRYNTH COOCLOCK
1,10000,TEN THOUSAND DRAGON
2,27551,LIMIT REVERSE
3,32864,THE 13TH GRAVE
4,35699,SPYRAL SLEEPER
...,...,...
13275,501000002,KING OF DESTRUCTION - XEXEX
13276,501000003,QUEEN OF FATE - ETERNIA
13277,501000004,TESTAMENT OF THE ARCANE LORDS
13278,501000006,"CHIMAERA, THE MASTER OF BEASTS"


In [216]:
frame_correto = pd.merge(df_ban, cod_name, how='inner', on='card_name')
frame_correto['remarks'] = frame_correto['remarks'].fillna('')
frame_correto

,card_type,card_name,condition,remarks,cod
0,Monster/Effect,AGIDO THE ANCIENT SENTINEL,Forbidden,,62320425
1,Monster/Effect,"ALPHA, THE MASTER OF BEASTS",Forbidden,,73304257
2,Monster/Effect,AMANO-IWATO,Forbidden,,32181268
3,Monster/Effect,APOQLIPHORT TOWERS,Forbidden,,27279764
4,Monster/Effect,ARCHLORD KRISTYA,Forbidden,,59509952
...,...,...,...,...,...
553,Trap,CYBERNETIC OVERFLOW,Unlimited,,82428674
554,Trap,MAGICAL EXPLOSION,Unlimited,,32723153
555,Trap,ULTIMATE CRYSTAL MAGIC,Unlimited,,36328300
556,Trap,VIRTUAL WORLD GATE - CHUCHE,Unlimited,,13364097


In [217]:
DESAJUSTADOS = df_ban.loc[~df_ban['card_name'].isin(frame_correto['card_name'])].reset_index(drop=True)

In [218]:
if len(DESAJUSTADOS.index):
    print('Problemas')
    print(DESAJUSTADOS)

In [219]:
cartas_banlist = frame_correto[['cod', 'condition', 'remarks']]
cartas_banlist

,cod,condition,remarks
0,62320425,Forbidden,
1,73304257,Forbidden,
2,32181268,Forbidden,
3,27279764,Forbidden,
4,59509952,Forbidden,
...,...,...,...
553,82428674,Unlimited,
554,32723153,Unlimited,
555,36328300,Unlimited,
556,13364097,Unlimited,


In [220]:
banlist = pd.merge(cartas_banlist, df_com, how='inner', on='cod')
banlist['cod'] = banlist['cod'].astype(str)
banlist

,cod,condition,remarks,card_name,tipo,effect,arquetype
0,62320425,Forbidden,,Agido the Ancient Sentinel,[Effect / Monster],If a card(s) is sent from the hand or Deck to ...,generic
1,73304257,Forbidden,,"Alpha, the Master of Beasts",[Effect / Monster],Cannot be Normal Summoned/Set. Must first be S...,generic
2,32181268,Forbidden,,Amano-Iwato,[Spirit / Monster],Cannot be Special Summoned. Monsters cannot ac...,generic
3,27279764,Forbidden,,Apoqliphort Towers,[Effect / Monster],"Cannot be Special Summoned. Requires 3 ""Qli"" T...",Apoqliphort | Qli
4,59509952,Forbidden,,Archlord Kristya,[Effect / Monster],If you have exactly 4 Fairy monsters in your G...,generic
...,...,...,...,...,...,...,...
553,82428674,Unlimited,,Cybernetic Overflow,Trap,"Banish ""Cyber Dragon(s)"" with different Levels...",Cybernetic | Cyber Dragon
554,32723153,Unlimited,,Magical Explosion,Trap,Activate only while you have no cards in your ...,generic
555,36328300,Unlimited,,Ultimate Crystal Magic,Trap,"When your ""Crystal Beast"" monster is destroyed...",Crystal Beast | Crystal | Ultimate Crystal
556,13364097,Unlimited,,Virtual World Gate - Chuche,Trap,You can target 1 face-up card on the field; sh...,Virtual World | Virtual World Gate


In [221]:
banlist.to_excel('banlist_complet_union.xlsx', index=False)